# Book review and ratings by user

In [166]:
import goodreads_api_client as gr
from goodreads import client
import pickle
import requests
from bs4 import BeautifulSoup
import xmltodict
import pprint
import json
import os
import re
import pandas as pd
import numpy as np
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

**Goodreads API credentials and wrappers**

In [167]:
api_key = 'Jjd519OAM82QfNQGepZsw'
api_secret = 'JY9m0mBvG7U1Ws4AUkpMucbwCNi6tndGTIqdASYo'

client = gr.Client(developer_key=api_key)

from goodreads import client
gc = client.GoodreadsClient(api_key, api_secret)

**Define functions to access APIs and scrape**

In [168]:
def get_info(url):
    key = api_key
    response = requests.get(url, params={'key': api_key})
    #soup = BeautifulSoup(response.content, "lxml-xml")
    results = xmltodict.parse(response.content)
    #results = json.dumps(results)
    return(results['GoodreadsResponse'])

def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #print(soup.prettify())
    return(soup)

**List of book id's on fantasy genre lists**

In [13]:
with open("book_ids.pkl", "rb") as f:
    fantasy_ids = pickle.load(f)
fantasy_ids = list(set(fantasy_ids))

**Define a function to get data for top reviews of a book by id**

In [169]:
def get_book_review_dict_by_user(i):
    d = {}
#for i in fantasy_ids[0:1]:
    d[i] = {}

    url = 'https://www.goodreads.com/book/show/'+i+'.xml'

    soup = get_soup(url)
    review_data = soup.find_all("div", {"class" :"review"})

    for r in review_data:
        try:
            userid = r.find("a", {"class" :"user"})['href'].split('show/')[1].split('-')[0]
            d[i][userid] = {}
            try:
                d[i][userid]['user'] = r.find("a", {"class" :"user"}).text
            except:
                pass
            try:
                d[i][userid]['review'] = r.find("div", {"class" :"reviewText"}).text.strip()
            except:
                pass
            try:
                d[i][userid]['rating'] = r.find("span", {"class" :"staticStars"}).text
            except:
                pass
                #d[i]['user_page'] = r.find("a", {"class" :"user"})['href']
            try:
                d[i][userid]['review_date'] = r.find("a", {"class" :"reviewDate"}).text            
            except:
                pass
        except:
            pass  
    return(d)

**Iterate through list of book id's. Stop every 1000 items to pickle the results. Save the final iteration variable as well**

In [14]:
len(fantasy_ids)

46057

**Reviews for first 10000 books**

In [20]:
for count, bookid in enumerate(fantasy_ids[0:10001]): 
    if count == 0:
        print(bookid)
    else:
        if count % 1000 == 0:
            print(count, bookid)

1138475
1000 18490893
2000 36284387
3000 16064396
4000 47601
5000 15756383
6000 274056
7000 25427978
8000 14656597
9000 17693317
10000 29921910


In [30]:
book_review_dict = get_book_review_dict_by_user(fantasy_ids[0])

iters = 0
for count, bookid in enumerate(fantasy_ids[0:10001]): 
    if count == 0:
        pass
    else:
        book_review_dict.update(get_book_review_dict_by_user(bookid))
    # Every 1000 books stop and pickle the dictioary
        if count % 1000 == 0:
            with open('book_reviews_by_user.pkl', 'wb') as f:
                pickle.dump(book_review_dict, f, pickle.HIGHEST_PROTOCOL)
            print(count)
            iters = count
            with open('book_review_iter_status.pkl', 'wb') as f:
                pickle.dump(iters, f)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [177]:
with open('book_reviews_by_user.pkl', 'wb') as f:
    pickle.dump(book_review_dict, f, pickle.HIGHEST_PROTOCOL)

In [179]:
test = pd.read_pickle('book_reviews_by_user.pkl')

In [181]:
len(book_review_dict)

26001

**Remaining 30k+ reviews**

In [ ]:
#book_review_dict = pd.read_pickle('book_reviews_by_user.pkl')

iters = len(book_review_dict)

for count, bookid in enumerate(fantasy_ids[iters:]): 
    if count == 0:
        pass
    else:
        book_review_dict.update(get_book_review_dict_by_user(bookid))
    # Every 1000 books stop and pickle the dictioary
        if count % 1000 == 0:
            with open('book_reviews_by_user.pkl', 'wb') as f:
                pickle.dump(book_review_dict, f, pickle.HIGHEST_PROTOCOL)
            print(count+15000)
            iters = count
            with open('book_review_iter_status.pkl', 'wb') as f:
                pickle.dump(iters, f)


16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [172]:
len(book_review_dict)

26001

In [164]:
list(book_review_dict.keys())[-1], fantasy_ids[26001]

('11073149', '11073149')

**Rearrange dictionary structure for mondgodb**

In [140]:
# Turn dictionary into a list
book_reviews_10k = []
for k, v in book_review_dict.items():
    book_reviews_10k.append({'bookid': k, 'reviews': v})

# Move userid to inside the review data rather than key to review data
reviews = []
for i, b in enumerate(book_reviews_10k):

    for userid in book_reviews_10k[i]['reviews'].keys():
        
        book_reviews_10k[i]['reviews'][userid]['userid'] = userid
        
# Final list of dictionaries
book_reviews = []

fix_iters = 0

for i, book in enumerate(book_reviews_10k):
    reviews = []
    for review in book['reviews'].values():
        reviews.append(review)
    book_reviews.append({'bookid': book['bookid'], 'reviews': reviews})
        
    # Every 1000 books stop and pickle
    if i % 1000 == 0:
        with open('book_reviews_dict_first10k.pkl', 'wb') as f:
            pickle.dump(book_reviews, f, pickle.HIGHEST_PROTOCOL)
        iters = count
        with open('book_review_fix_iter_status.pkl', 'wb') as f:
            pickle.dump(fix_iters, f)


**END NOTEBOOK**

***